In [1]:
import dspy
import pydantic
import openai
print(dspy.__version__)
print(pydantic.__version__)
print(openai.__version__)

3.0.0b1
2.11.7
1.88.0


In [2]:
# Config setup
import os
from dotenv import load_dotenv

load_dotenv()

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
OPENROUTER_BASE_URL = os.getenv("OPENROUTER_BASE_URL")
MODEL = os.getenv("MODEL_NAME")
TEMPERATURE = os.getenv("TEMPERATURE")
MAX_TOKENS = os.getenv("MAX_TOKENS")

print(MODEL)
print(TEMPERATURE)
print(MAX_TOKENS)

openrouter/google/gemini-2.5-flash-lite-preview-06-17
1.0
4000


In [3]:
lm = dspy.LM(model=MODEL)

In [4]:
import dspy
dspy.configure(lm=lm)
pred = dspy.Predict('query -> answer: str')

In [6]:
pred(query="hello world how r u?")

Prediction(
    answer='Hello world! I am doing well, thank you for asking. How are you today?'
)

In [9]:
# ---------- Minimal yet complete data-model scaffold (Pydantic) ----------

from enum import Enum
from typing import List, Dict, Optional, Literal
from pydantic import BaseModel, Field


class QueryAnalysis(BaseModel):
    """Analyze and break down the user's prompt to make sure you fully understand it. """
    query_type: Literal["depth_first", "breadth_first", "straightforward"] = Field(description="The type of query to be answered")
    complexity: Literal["simple", "medium", "complex"] = Field(description="The complexity of the query")
    main_concepts: List[str] = Field(description="Key concepts to research")
    key_entities: List[str] = Field(description="Important entities mentioned")
    relationships: List[str] = Field(description="Relationships in the task")
    notes: Optional[str] = Field(description="Any temporal or contextual constraints on the question")
    answer_format: Optional[str] = Field(description="The recommended format of the answer such as detailed report, a list of entities, an anlysis of different perspectives, etc.")


# 2) Lead-agent plan ------------------------------------------------------
class PlanStep(BaseModel):
    id: int
    description: str
    depends_on: List[int] = Field(default_factory=list)       # other step IDs
    budget_calls: int = 5                                     # expected tool calls

class ResearchPlan(BaseModel):
    steps: List[PlanStep]

# # 3) Sub-agent delegation & results --------------------------------------
# class SubagentTask(BaseModel):
#     id: int
#     step_id: int                                              # link back to PlanStep
#     objective: str
#     expected_output: Optional[str] = None
#     status: Literal["pending", "in_progress", "completed", "failed"] = "pending"

# class SubagentResult(BaseModel):
#     task_id: int
#     findings: str
#     summary: str
#     sources: List[str] = Field(default_factory=list)
#     source_quality: Optional[Literal["high", "medium", "low"]] = None
#     notes: Optional[str] = None

# # 4) Final synthesis ------------------------------------------------------
# class ResearchReport(BaseModel):
#     query: str
#     synthesis: str
#     analyses: Dict[str, str] = Field(default_factory=dict)
#     summary: str
#     total_tool_calls: int
#     gap_analysis: str

In [21]:
# DSPY SIGNATURE

class AnalyzeQuery(dspy.Signature):
    """Analyze a user query to determine research approach."""
    query: str = dspy.InputField(desc="The user's research query")
    analysis: QueryAnalysis = dspy.OutputField(desc="Comprehensive query analysis")

class PlanResearch(dspy.Signature):
    """Create a research plan based on query analysis."""
    query: str = dspy.InputField(desc="The user's research query")
    analysis: QueryAnalysis = dspy.InputField(desc="Comprehensive query analysis")
    plan: ResearchPlan = dspy.OutputField(desc="Detailed research plan")

In [22]:
import dspy




# class MultiAgentSystem(dspy.Module):
#     def __init__(self):
#         self.planner = dspy.ChainOfThought()

query_analysis = dspy.ChainOfThought(AnalyzeQuery)
research_plan = dspy.ChainOfThought(PlanResearch)

In [17]:

pred = query_analysis(query = "Compare all the major frontend frameworks based on performance, learning curve, ecosystem, and industry adoption")

In [27]:
# Use the research_plan module that was already created
plan_result = research_plan(query="What is the capital of France?", analysis=pred.analysis)